#### This notebook is the main notebook to run the recommendations of the procedures based on the lab results and diagnosis

## Load the data

In [51]:
import numpy as np
import pandas as pd


# trin and test output
data_test = pd.read_csv('data_processing/data_test.csv')
data_train = pd.read_csv('data_processing/data_train.csv')

PC_pr_tr = data_train[[data_train.columns[0]]+ list(data_train.columns[314: 375])].sort_values('PC_enc')
PC_pr_ts = data_test[[data_test.columns[0]]+ list(data_test.columns[314: 375])].sort_values('PC_enc')



#df_test = pd.read_csv('data/data_test.csv').sort_values('PC_enc')
SP_pr_tr = data_train[[data_train.columns[0]]+ list(data_train.columns[407:])].sort_values('PC_enc')
SP_pr_ts = data_test[[data_test.columns[0]]+ list(data_test.columns[407:])].sort_values('PC_enc')
print(SP_pr_tr.head())




# Predictions from model 1 and 2

# train set
df_model2_train = pd.read_csv('evaluation/data/CF_AE_pred_train.csv').sort_values('PC_enc')
df_model1_train = pd.read_csv('evaluation/data/Model1_pred_train.csv').sort_values('PC_enc')
#print(df_model1_train.shape)
#print(df_model1_train[0])

df_model2_train = pd.merge(df_model2_train, df_model1_train['PC_enc'], on='PC_enc', how='inner').sort_values('PC_enc')



# test set
df_model2_pred = pd.read_csv('evaluation/data/CF_AE_pred.csv').sort_values('PC_enc')
df_model1_pred = pd.read_csv('evaluation/data/Model1_pred.csv').sort_values('PC_enc')

# privoder information
SP_id_tr = data_train[[data_train.columns[0]]+ list(data_train.columns[377: 407])].sort_values('PC_enc')
SP_id_ts = data_test[[data_test.columns[0]]+ list(data_test.columns[377: 407])].sort_values('PC_enc')

#print(df_model1_train[:10,:])
#print(df_model2_train[:10,:])


X_train = pd.merge( df_model2_train, df_model1_train, on='PC_enc', how='inner').to_numpy()[:,1:]
#X_train = pd.merge( train_joined, SP_id_tr, on='PC_enc', how='inner').to_numpy()[:,1:] # Add prov info

#print(train_joined[:10,:])

#X_train =  pd.merge(SP_id_tr, df_model1_train, on='PC_enc', how='inner').sort_values('PC_enc').to_numpy()[:,1:]


#X_train =  SP_pr_tr.sort_values('PC_enc').to_numpy()[:,1:]
#################################


X_test  = pd.merge( df_model2_pred, df_model1_pred, on='PC_enc', how='inner').to_numpy()[:,1:]
#X_test = pd.merge( test_joined, SP_id_ts, on='PC_enc', how='inner').to_numpy()[:,1:] # Add prov info

#X_test =   pd.merge(SP_id_ts , df_model1_pred, on='PC_enc', how='inner').sort_values('PC_enc').to_numpy()[:,1:]



#
#X_test =   SP_pr_ts.sort_values('PC_enc').to_numpy()[:,1:] 


#print(X_train.shape)

#print(X_train[0])
#print(X_test[:10,:])
# Only include model 2 as input
#X_train = df_model1_train.to_numpy()[:,1:]
#X_test = df_model1_pred.to_numpy()[:,1:]


Y_train = SP_pr_tr.sort_values('PC_enc').to_numpy()[:,1:]
Y_test = SP_pr_ts.sort_values('PC_enc').to_numpy()[:,1:]


d = X_train.shape[1]
N_train = X_train.shape[0]

d_out = Y_train.shape[1]
print(X_train.shape)
print(Y_train.shape)

#print(X_train[:10]!=Y_train[:10])
#print(X_test[:10]!=Y_test[:10])

        PC_enc  F0_y_y  F1  F2  F3  F4  F5  F6  F7  F8  ...  F51_y  F52_y  \
4326  16116015       0   0   0   0   0   0   0   0   0  ...      0      0   
1525  16175218       0   0   0   0   0   0   0   0   0  ...      0      0   
5042  16494386       0   0   0   0   0   0   0   0   0  ...      0      0   
5055  16927888       0   0   0   0   0   0   0   0   0  ...      0      0   
2418  17091694       1   0   0   0   1   1   0   0   0  ...      0      0   

      F53_y  F54_y  F55_y  F56_y  F57_y  F58_y  F59_y  F60_y  
4326      0      0      0      0      0      0      0      0  
1525      0      0      0      0      0      0      0      0  
5042      0      0      0      0      0      0      0      0  
5055      0      0      0      0      0      0      0      0  
2418      0      0      0      0      0      0      0      0  

[5 rows x 62 columns]
(5500, 122)
(5500, 61)


In [52]:
## test and debug: REMOVE LATER
print(pd.merge(df_model1_pred, df_model2_pred, on='PC_enc', how='inner').PC_enc.tail(10))
print(SP_pr_ts.PC_enc.tail(10))


1311    131240872227
1312    131240887260
1313    131240890527
1314    131240893153
1315    131240899094
1316    131240966487
1317    131241025125
1318    131241061358
1319    131241077737
1320    131261087497
Name: PC_enc, dtype: int64
907     131240872227
906     131240887260
893     131240890527
870     131240893153
879     131240899094
874     131240966487
889     131241025125
886     131241061358
884     131241077737
1320    131261087497
Name: PC_enc, dtype: int64


#### We'll start simple, with a single fully-connected neural layer as encoder and as decoder

## Model 1

In [69]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

#input_img = Input(shape=(d,))
#encoded = Dense(100, activation='relu')(input_img)
#decoded = Dense(d_out, activation='relu')(encoded)
p = 0.35 # dropout

input_img = Input(shape=(d,))
encoded = Dense(200, activation='relu')(input_img)
encoded = Dropout(rate=p)(encoded)
encoded = Dense(120, activation='relu')(encoded)
encoded = Dropout(rate=p)(encoded)
encoded = Dense(100, activation='relu')(encoded)
encoded = Dropout(rate=p)(encoded)
#encoded = Dense(80, activation='relu')(encoded)
#encoded = Dropout(rate=p)(encoded)
encoded = Dense(80, activation='relu')(encoded)
encoded = Dropout(rate=p)(encoded)
decoded = Dense(d_out, activation='relu')(encoded)

#encoding_dim = 40

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

#### Now let's train our autoencoder

In [70]:
import keras
#opt = keras.optimizers.SGD(lr=9e-2, momentum=0.9, decay=1e-2/100)
sgd = keras.optimizers.SGD(lr=0.06, decay=1e-4, momentum=0.9) #(lr=0.03, decay=1e-6, momentum=0.9)
#sgd=keras.optimizers.Adam(learning_rate=0.01)
autoencoder.compile(optimizer=sgd, loss = 'mean_squared_error') #loss='mean_squared_error')

#### Now let's train our autoencoder for 50 epochs:

In [71]:

autoencoder.fit(X_train, Y_train,
                epochs=250,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, Y_test))#,class_weight=weights) 

Train on 5500 samples, validate on 1321 samples
Epoch 1/250
5500/5500 [==============================] - 0s 47us/step - loss: 0.0734 - val_loss: 0.0712
Epoch 2/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0675 - val_loss: 0.0679
Epoch 3/250
5500/5500 [==============================] - 0s 14us/step - loss: 0.0644 - val_loss: 0.0635
Epoch 4/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0606 - val_loss: 0.0604
Epoch 5/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0586 - val_loss: 0.0592
Epoch 6/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0578 - val_loss: 0.0587
Epoch 7/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0574 - val_loss: 0.0584
Epoch 8/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0572 - val_loss: 0.0582
Epoch 9/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0570 - val_loss: 0.0580
Epoch 10/250
550

5500/5500 [==============================] - 0s 15us/step - loss: 0.0562 - val_loss: 0.0575
Epoch 79/250
5500/5500 [==============================] - 0s 17us/step - loss: 0.0562 - val_loss: 0.0575
Epoch 80/250
5500/5500 [==============================] - 0s 14us/step - loss: 0.0562 - val_loss: 0.0575
Epoch 81/250
5500/5500 [==============================] - 0s 14us/step - loss: 0.0562 - val_loss: 0.0575
Epoch 82/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0562 - val_loss: 0.0575
Epoch 83/250
5500/5500 [==============================] - 0s 15us/step - loss: 0.0562 - val_loss: 0.0575
Epoch 84/250
5500/5500 [==============================] - 0s 14us/step - loss: 0.0562 - val_loss: 0.0575
Epoch 85/250
5500/5500 [==============================] - 0s 14us/step - loss: 0.0562 - val_loss: 0.0575
Epoch 86/250
5500/5500 [==============================] - 0s 14us/step - loss: 0.0562 - val_loss: 0.0575
Epoch 87/250
5500/5500 [==============================] - 0s 13us/st

5500/5500 [==============================] - 0s 13us/step - loss: 0.0562 - val_loss: 0.0574
Epoch 156/250
5500/5500 [==============================] - 0s 12us/step - loss: 0.0561 - val_loss: 0.0574
Epoch 157/250
5500/5500 [==============================] - 0s 12us/step - loss: 0.0562 - val_loss: 0.0574
Epoch 158/250
5500/5500 [==============================] - 0s 12us/step - loss: 0.0562 - val_loss: 0.0574
Epoch 159/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0562 - val_loss: 0.0574
Epoch 160/250
5500/5500 [==============================] - 0s 12us/step - loss: 0.0562 - val_loss: 0.0574
Epoch 161/250
5500/5500 [==============================] - 0s 12us/step - loss: 0.0562 - val_loss: 0.0574
Epoch 162/250
5500/5500 [==============================] - 0s 12us/step - loss: 0.0561 - val_loss: 0.0574
Epoch 163/250
5500/5500 [==============================] - 0s 12us/step - loss: 0.0562 - val_loss: 0.0574
Epoch 164/250
5500/5500 [==============================] - 0

5500/5500 [==============================] - 0s 14us/step - loss: 0.0561 - val_loss: 0.0574
Epoch 233/250
5500/5500 [==============================] - 0s 14us/step - loss: 0.0561 - val_loss: 0.0574
Epoch 234/250
5500/5500 [==============================] - 0s 14us/step - loss: 0.0561 - val_loss: 0.0574
Epoch 235/250
5500/5500 [==============================] - 0s 14us/step - loss: 0.0561 - val_loss: 0.0574
Epoch 236/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0561 - val_loss: 0.0574
Epoch 237/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0561 - val_loss: 0.0574
Epoch 238/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0561 - val_loss: 0.0574
Epoch 239/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0561 - val_loss: 0.0574
Epoch 240/250
5500/5500 [==============================] - 0s 13us/step - loss: 0.0561 - val_loss: 0.0574
Epoch 241/250
5500/5500 [==============================] - 0

### Predict

In [72]:

pred = autoencoder.predict(X_test)


print(pred.shape)


(1321, 61)


### back to dataframe

In [73]:
#print(SP_pr_ts['PC_enc'].head())

PC_enc_list = SP_pr_ts['PC_enc'].sort_values().tolist()
df_new = pd.DataFrame(data=pred, columns=SP_pr_ts.columns[1:])
PC_enc_list = SP_pr_ts['PC_enc'].sort_values().tolist()
df_new['PC_enc'] = PC_enc_list
df_new = df_new[['PC_enc']+ list(df_new.columns[0:-1])]
print(df_new.head())

#df_new = pd.concat([df_new, pd.DataFrame(data=pred, columns=SP_pr_ts.columns[1:])], axis=1)#.sort_values('PC_enc')

#print(df_new.head())

df_new.to_csv('evaluation/data/Model_ens_pred.csv',index=False)




         PC_enc    F0_y_y        F1        F2   F3        F4        F5  \
0  131023122942  0.546463  0.414371  0.220548  0.0  0.260261  0.217132   
1  131023156693  0.545240  0.413493  0.220841  0.0  0.260159  0.218522   
2  131023214462  0.546800  0.414542  0.220879  0.0  0.260450  0.216824   
3  131023214801  0.569976  0.433664  0.221181  0.0  0.265465  0.192587   
4  131023242322  0.548761  0.416177  0.220436  0.0  0.260848  0.214571   

         F6        F7        F8  ...  F51_y     F52_y  F53_y  F54_y  F55_y  \
0  0.141307  0.122093  0.115654  ...    0.0  0.022026    0.0    0.0    0.0   
1  0.141048  0.121528  0.115542  ...    0.0  0.022162    0.0    0.0    0.0   
2  0.141300  0.122159  0.115754  ...    0.0  0.022163    0.0    0.0    0.0   
3  0.143177  0.134154  0.114108  ...    0.0  0.021682    0.0    0.0    0.0   
4  0.141631  0.123166  0.115725  ...    0.0  0.022089    0.0    0.0    0.0   

   F56_y  F57_y  F58_y  F59_y  F60_y  
0    0.0    0.0    0.0    0.0    0.0  
1    0.0

### Debug

In [182]:
i = 3
print(X_test[i,:10])
print(pred[i,:10])



[0 0 0 0 1 1 0 1 0 0]
[0.00296615 0.         0.         0.         0.9133584  1.0397264
 0.         0.9226485  0.         0.03729805]


In [195]:
eta = 0.2

np1 = X_test#.to_numpy()[:,1:]
np2 = pred#.to_numpy()[:,1:]

i = 12

print(np.where((np1[i,:]>eta).astype(int)))
print(np.where((np2[i,:]>eta).astype(int)))

print(np1[i,:])
print(np2[i,:])

#r = get_performance(df_test,df_model_ens_pred, 0.3)
#print(r)

#print(df_test.to_numpy()[i])
#print(df_model_ens_pred.to_numpy()[i])
#print()

(array([ 0,  1,  8, 15, 29, 39]),)
(array([ 0,  1,  8, 15, 29, 39]),)
[1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0.98706645 1.0094048  0.         0.         0.         0.
 0.         0.         1.0362589  0.         0.         0.
 0.         0.         0.         0.99164975 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         1.1063786
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         1.0002358  0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]


In [203]:
print(df_new.head())
print(SP_pr_ts.sort_values('PC_enc').head())

         PC_enc    F0_y_y       F1        F2   F3        F4        F5  \
0  131091501475  0.000000  0.00000  0.981716  0.0  0.000000  0.000000   
1  131214560515  0.000000  0.00000  0.000000  0.0  0.953292  0.000000   
2  131091028338  0.000000  0.00000  0.961141  0.0  0.000000  0.028484   
3  131090937788  0.002966  0.00000  0.000000  0.0  0.913358  1.039726   
4  131213181257  0.989081  1.00959  0.000000  0.0  0.000000  0.000000   

         F6        F7        F8  ...  F51_y  F52_y  F53_y  F54_y  F55_y  \
0  0.018934  0.000000  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
1  0.938199  0.000000  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
2  0.943942  0.000000  0.037403  ...    0.0    0.0    0.0    0.0    0.0   
3  0.000000  0.922648  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
4  0.000000  0.000000  0.000000  ...    0.0    0.0    0.0    0.0    0.0   

   F56_y  F57_y  F58_y  F59_y  F60_y  
0    0.0    0.0    0.0    0.0    0.0  
1    0.0    0.0    0.0    0.0   